In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
# from langchain.llms import LlamaCpp
from langchain.llms import CTransformers

In [3]:
import json
creds = {}
with open("creds.json") as f:
    creds = json.load(f)

In [4]:
loader = PyPDFLoader(creds["filepath"])
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
print(docs[0].page_content)

embeddings = HuggingFaceEmbeddings()

Unit -III Map Reduce And Spark Framework
Map Reduce: Anatomy of a Map Reduce job Run, Failure, Job Scheduling, Shuffle and sort, 
Task Exception, Map Reduce Types and formats, Map Reduce feactures, Map Reduce 
programming:  I/O formats, Map side join, Reduce side join,Secondary sorting, Pipelining Map 
reduce jobs.
Spark frameworks:  Introduction to GPU Computing,CUDA programming model, CUDA API, 
Simple matrix,Multiplication in CUDA, CUDA memory model, Shared memory matrix 
multiplication,Additional CUDA API Features.
Anatomy of a Map Reduce job Run
The term "big data analytics" refers to the application of cutting-edge analytical methods to very 
large, diversified big data sets, which may comprise structured, semi-structured, and unstructured 
data from many sources and range in size from terabytes to zettabytes. The high volume, high 
velocity, and high variety of big data are its characteristics. Due to the influence of artificial 
intelligence (AI), mobile technology, social medi

In [7]:
# create vector store if not exists
db = Chroma.from_documents(docs, embeddings)
db.persist()

In [8]:
retriever = db.as_retriever()

In [5]:
llm = CTransformers(
    model=creds["modelpath"],
    model_type="llama",
    # callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

In [20]:
from langchain import PromptTemplate

template = """
You are a big data analytics expert. These Human will ask you a questions and doubts on the topic. 
Use following piece of context to answer the question. 
If you don't know the answer, just say you don't know. 
Keep the answer within 5 sentences and concise.

Context: {context}
Question: {question}
Answer: 

"""

prompt = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

In [21]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"context": retriever,  "question": RunnablePassthrough()} 
  | prompt 
  | llm
  | StrOutputParser() 
)

In [ ]:
qn = input("What is it you want? >> ")
res = rag_chain.invoke(qn)
print(res)

Number of tokens (3336) exceeded maximum context length (512).
